In [ ]:
! pip install requests
! pip install ntplib

In [ ]:
#github up用

#bitFlyer Lightning APIと用いて自動売買するためのコードです

#１分ごとにビットコインFXの価格を取得しています


#購入（仕掛け）条件が発注される時は、active注文なし、建玉なし、現在の価格が直近２０分の価格で最高値であった時

#購入（仕掛け）発注の中身は、指値注文、BUY、価格は現在の価格、注文数量は0.1、期限切れまでの時間は１０分


#手仕舞い条件が発注される時は、active注文なし、建玉あり、現在の価格が直近２０分の価格で最安値であった時

#手仕舞い発注の中身は、成行注文、SELL、注文数量は0.1



#実行に際して、	API Key、API Secretを書き換える必要があります。

#現在の価格の最高値（最安値）判定の時間数を、仕掛けと手仕舞いで異なる設定が簡単にできるようなコードにしています。


import requests
import time
import requests
import json
import time
import hmac
import hashlib
    

def get_active_child_orders():
    path_url = "/v1/me/getchildorders?product_code=FX_BTC_JPY&child_order_state=ACTIVE"
    method = "GET"
    timestamp = int(time.time() * 1000)
    timestamp = str(timestamp)
    message = timestamp + method + path_url
    signature = hmac.new(bytearray('API Secret'.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()

    headers = {
            'ACCESS-KEY' : 'API Key',
            'ACCESS-TIMESTAMP' : timestamp,
            'ACCESS-SIGN' : signature,
            'Content-Type' : 'application/json'}

    response = requests.get(url + path_url, headers = headers)
    return json.loads(response.text)


def get_positions():
    url = "https://api.bitflyer.com"
    path_url = "/v1/me/getpositions?product_code=FX_BTC_JPY"
    method = "GET"
    timestamp = int(time.time() * 1000)
    timestamp = str(timestamp)
    message = timestamp + method + path_url
    signature = hmac.new(bytearray('API Secret'.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()

    headers = {
            'ACCESS-KEY' : 'API Key',
            'ACCESS-TIMESTAMP' : timestamp,
            'ACCESS-SIGN' : signature,
            'Content-Type' : 'application/json'}

    response = requests.get(url + path_url, headers = headers)
    return json.loads(response.text)


def get_bitcoin_price():
    url = "https://api.bitflyer.com/v1/getticker"
    params = {'product_code': 'FX_BTC_JPY'}
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print("APIリクエストに失敗しました。ステータスコード:", response.status_code)
        return None
  
    try:
        data = response.json()
        return data['ltp']
    except Exception as e:
        print("JSONデータの解析に失敗しました。エラー:", e)
        return None


url = "https://api.bitflyer.com"
path_url = "/v1/me/sendchildorder"
method = "POST"



if __name__ == "__main__":
    price_list = []
    interval = 2  # 価格データを取得する間隔（秒）

    buy_min_interval=20 #何分ぶりの新高値なのか？(購入用)
    sell_min_interval=20 #何分ぶりの新安値なのか？(売却用)

    while True:
        start_time = time.time()

        price = get_bitcoin_price()
        if price is not None:                                            #価格の取得の有無
            price_list.append(price)
            print("ビットコインの現在の価格は:", price, "円です。")
            print(price_list)
            current_time = time.strftime("%H:%M:%S")
            print("Current time:", current_time)

            if len(price_list) >= buy_min_interval and len(price_list) >= sell_min_interval: #sell_min_intervalの最安値の計算ができるようになるまで待つ
              buy_min_list = price_list[-buy_min_interval:]
              buy_maxprice = max(buy_min_list)
 

              sell_min_list = price_list[-sell_min_interval:]
              sell_minprice = min(sell_min_list)

              positions = get_positions()  #辞書リスト形式

              active_child_orders = get_active_child_orders()

              if len(active_child_orders) > 0:                          #ACTIVEな注文の有無の判定
                for order in active_child_orders:
                      print("Active 注文: ", order)
              else:
                print("Active な注文がありません。")

                if len(positions) == 0:                                 #ポジションが有無の判定
                 print("建玉がありません。")
                 if buy_min_list[-1] == buy_maxprice:                              #現在の価格が過去２０分で最高値であるか判定
                   print("現在の価格は、",buy_min_interval,"ぶりの現在最高値です:",price)

                   timestamp = int(time.time() * 1000)#買い注文用のコード始
                   timestamp = str(timestamp)

                   payload = {
                       "product_code": "FX_BTC_JPY",
                       "child_order_type": "LIMIT",  #指値注文の場合は "LIMIT", 成行注文の場合は "MARKET"
                       "side": "BUY",                #買い注文の場合は "BUY", 売り注文の場合は "SELL" 
                       "price": price-10000,         #child_order_type に "LIMIT" を指定した場合は必須
                       "size": 0.01,                 #注文数量を指定します。
                       "minute_to_expire": 10,       #期限切れまでの時間を分で指定します。省略した場合の値は 43200 (30 日間) です。
                       "time_in_force": "GTC"        # 執行数量条件 を "GTC", "IOC", "FOK" のいずれかで指定します。省略した場合の値は "GTC" です。
                   }

                   body = json.dumps(payload) #payloadをjson形式へ変換
                   message = timestamp + method + path_url + body
                   signature = hmac.new(bytearray('API Secret'.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()

                   headers = {
                           'ACCESS-KEY' : 'API Key',
                           'ACCESS-TIMESTAMP' : timestamp,
                           'ACCESS-SIGN' : signature,
                           'Content-Type' : 'application/json'}

                   response = requests.post( url + path_url , headers = headers, data=body)

                   print(response.text)#買い注文コード終

                else:                                                    #売り用のコード　ポジションが有る時
                 print("建玉があります。")
                 for position in positions:  #辞書形式に
                    size = position['size']
                    print("建玉のサイズ:", size)

                 if sell_min_list[-1] == sell_minprice:                       #現在の価格が過去２０分で最高値であるか判定
                   print("現在の価格は、",sell_min_interval,"ぶりの現在最小値です:",price)

                   timestamp = int(time.time() * 1000)#売り（手仕舞い）注文始
                   timestamp = str(timestamp)

                   payload = {
                       "product_code": "FX_BTC_JPY",
                       "child_order_type": "MARKET",  #指値注文の場合は "LIMIT", 成行注文の場合は "MARKET"
                       "side": "SELL",                #買い注文の場合は "BUY", 売り注文の場合は "SELL" 
#                       "price": price,              #child_order_type に "LIMIT" を指定した場合は必須
                       "size": 0.01,                 #注文数量を指定します
#                      "minute_to_expire": 10,       #期限切れまでの時間を分で指定します。省略した場合の値は 43200 (30 日間) です。
                       "time_in_force": "GTC"        # 執行数量条件 を "GTC", "IOC", "FOK" のいずれかで指定します。省略した場合の値は "GTC" です。
                   }

                   body = json.dumps(payload) #payloadをjson形式へ変換
                   message = timestamp + method + path_url + body
                   signature = hmac.new(bytearray('API Secret'.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()

                   headers = {
                           'ACCESS-KEY' : 'API Key',
                           'ACCESS-TIMESTAMP' : timestamp,
                           'ACCESS-SIGN' : signature,
                           'Content-Type' : 'application/json'}

                   response = requests.post( url + path_url , headers = headers, data=body)

                   print(response.text)#売り（手仕舞い）注文終



            if len(price_list) >= 1000:#プライスリストの長さの制限
                del price_list[0]
        else:
            print("価格情報の取得に失敗しました。")

        elapsed_time = time.time() - start_time
        sleep_time = max(0, interval - elapsed_time) #念のため、スリープタイムがマイナスにならないように
        time.sleep(sleep_time)
